In [2]:
import re
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import pyBigWig as pbg
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy.stats
from scipy import sparse

In [3]:
def ohe(seq):
    seq2=list()
    mapping = {"A":[1, 0, 0, 0], "C": [0, 1, 0, 0], "G": [1, 0, 0, 0], "T":[0, 0, 0, 1]}
    for i in seq:
      seq2.append(mapping[i]  if i in mapping.keys() else [0, 0, 0, 0]) 
    return np.array(seq2)

In [4]:
#fasta to ohe
f=open('/home/florian/projet/r6.16/dmel-all-chromosome-r6.16.fasta','r')
os.chdir('/home/florian/projet/r6.16')
chr_name = str()
seq=[]
D={}
for line in f:
        line = line.rstrip()
        if line[0] == '>':
                if line[1:3]=="2C":
                        break
                if seq != []:
                        D[chr_name]=ohe(seq)
                chr_name = line[1:3]
                seq = []
        else:
                seq += line
D["Y"]=ohe(seq)
np.savez_compressed('seq',**D)

In [5]:
#bigwig to numpy
os.chdir('/home/florian/projet/scATACseq')
fichiers=os.listdir('/home/florian/projet/scATACseq')
valeurs= []
chr=["2L","2R","3R","3L","4","X","Y"]
Dic={"2L"}
for file in fichiers:
    #print("processing file", file)
    bw=pbg.open(file)
    array = np.concatenate((bw.values("2L",0,-1,numpy=True),bw.values("2R",0,-1,numpy=True),bw.values("3R",0,-1,numpy=True),bw.values("3L",0,-1,numpy=True),bw.values("4",0,-1,numpy=True),bw.values("X",0,-1,numpy=True),bw.values("Y",0,-1,numpy=True)))
    if len(valeurs) == 0:
        valeurs = array
    else:
        valeurs += array


In [21]:
#bigwig to numpy
os.chdir('/home/florian/projet/scATACseq')
fichiers=os.listdir('/home/florian/projet/scATACseq')
valeurs, mins, maxs = [], [], []
chr=["2L","2R","3R","3L","4","X","Y"]
Dic={"2L"}
for file in fichiers:
    #print("processing file", file)
    bw=pbg.open(file)
    array = np.concatenate((bw.values("2L",0,-1,numpy=True),bw.values("2R",0,-1,numpy=True),bw.values("3R",0,-1,numpy=True),bw.values("3L",0,-1,numpy=True),bw.values("4",0,-1,numpy=True),bw.values("X",0,-1,numpy=True),bw.values("Y",0,-1,numpy=True)))
    mins.append(array.min())
    maxs.append(array.max())
    if len(valeurs) == 0:
        valeurs = array
    else:
        valeurs += array

137547960

In [28]:
os.chdir('/home/florian/projet/scATACseq')
np.savez_compressed('scATACseq',valeurs=valeurs)

In [29]:
np.load('/home/florian/projet/scATACseq/scATACseq.npz')['valeurs']

array([ 0.      ,  0.      ,  1.429383, ..., 64.00831 , 39.947495,
       23.297546], dtype=float32)

In [ ]:
plt.figure(figsize=(5,4))
plt.hist(val,bins=500, density=True)
plt.title("densité")
#plt.xscale('log')
#plt.yscale('log')
plt.show()

In [ ]:
plt.plot((val[1000000:1100000]))

In [6]:
val=np.copy(valeurs)
val[val >= 2000] = 2000
val=val/2000

In [ ]:
np.arange(10)[4::-1]

In [ ]:
#bigwig to sparse
os.chdir('/home/florian/projet/scATACseq')
fichiers=os.listdir('/home/florian/projet/scATACseq')
values=[]
for file in fichiers:
    bw=pbg.open(file)
    array=np.concatenate((bw.values("4",0,-1,numpy=True),bw.values("X",0,-1,numpy=True),bw.values("Y",0,-1,numpy=True),bw.values("2L",0,-1,numpy=True),bw.values("2R",0,-1,numpy=True),bw.values("3L",0,-1,numpy=True),bw.values("3R",0,-1,numpy=True)))
    values.append(sparse.csr_matrix(array))


In [7]:
#create dataset
Ytrain_chr2L=np.copy(val[1000:11000])#23512712])
chr2L=np.copy(np.load('/home/florian/projet/r6.16/seq.npz')['2L'])
Xtrain_chr2L=[]
for i in range(1000,11000):#len(chr2L)-1001):
    Xtrain_chr2L.append(chr2L[i-1000:i+1001])
Xtrain_chr2L = np.array(Xtrain_chr2L)


In [ ]:
#modèle

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(128, kernel_size=(6), activation='relu', input_shape=(2001,4)),
    tf.keras.layers.MaxPooling1D(pool_size=(12)),
    tf.keras.layers.Conv1D(64, kernel_size=(12), activation='relu'),
    #tf.keras.layers.MaxPooling1D(pool_size=(12)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='softmax')
    ])

loss_fn = tf.keras.losses.MeanAbsoluteError()

model.compile(loss=loss_fn,
    optimizer='adam',
    metrics=['mae'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1996, 128)         3200      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 166, 128)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 155, 64)           98368     
                                                                 
 flatten (Flatten)           (None, 9920)              0         
                                                                 
 dense (Dense)               (None, 128)               1269888   
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                        

In [13]:
history = model.fit(Xtrain_chr2L, Ytrain_chr2L, epochs=2, validation_split=0.30, verbose=1)

Epoch 1/2
219/219 [==============================] - 7s 31ms/step - loss: 0.9416 - mae: 0.9416 - val_loss: 0.9943 - val_mae: 0.9943
Epoch 2/2
219/219 [==============================] - 6s 29ms/step - loss: 0.9416 - mae: 0.9416 - val_loss: 0.9943 - val_mae: 0.9943


In [ ]:
len(Xtrain_chr2L)==len(Ytrain_chr2L)

In [22]:
len((np.load('/home/florian/projet/r6.16/seq.npz')['2L']))+len(np.load('/home/florian/projet/r6.16/seq.npz')['2R'])+len(np.load('/home/florian/projet/r6.16/seq.npz')['3R'])+len(np.load('/home/florian/projet/r6.16/seq.npz')['3L'])+len(np.load('/home/florian/projet/r6.16/seq.npz')['4 '])+len(np.load('/home/florian/projet/r6.16/seq.npz')['X '])+len(np.load('/home/florian/projet/r6.16/seq.npz')['Y'])

137547960

In [34]:
model.predict(Xtrain_chr2L[0])

: 

: 

In [12]:
np.shape(Xtrain_chr2L[0])

(2001, 4)

In [ ]:
#create dataset
Ytrain_chr2L=np.copy(val[1000:11000])#23512712])
chr2L=np.copy(np.load('/home/florian/projet/r6.16/seq.npz')['2L'])
Xtrain_chr2L=[]
for i in range(1000,11000):#len(chr2L)-1001):
    Xtrain_chr2L.append(chr2L[i-1000:i+1001])
Xtrain_chr2L = np.array(Xtrain_chr2L)


In [ ]:
model.evaluate(test_features, test_labels, verbose=0)